In [2]:
import torch.optim as optim
from torch import nn
from gloss_dataset import GlossDataset
from gloss_model import GlossModel
from torch.utils.tensorboard import SummaryWriter


In [9]:
# get data for training
gd =GlossDataset() 
input_size=gd[0][0].shape[1]
class_no=len(gd)
input_size,class_no

(1596, 12)

In [10]:
# provide input and class size
model = GlossModel(input_size,class_no)
optim = optim.Adam(model.parameters(), lr=0.1)
loss_fn= nn.CrossEntropyLoss()
model

GlossModel(
  (softmax): Softmax(dim=None)
  (relu): ReLU()
  (lstm1): LSTM(1596, 128, batch_first=True)
  (lstm2): LSTM(128, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=12, bias=True)
)

In [11]:
# initialize sumamry writer
writer=SummaryWriter()

In [ ]:
# Start model training
model.train()

for i in range(epoch):
    optim.zero_grad()
    out = model(x_train)
    loss = loss_fn(out, y_train.argmax())
    loss.backward()
    optim.step()
    writer.add_scalar("Loss/epoch", loss.item(), i)

writer.close()


In [ ]:
# save the model
torch.save(model, "swaram_lstm.pt")
